# Black Jack with Monte Carlo
In this task you are asked to find an optimal policy for a Black Jack game. You are going to use an OpenAI Gym [Black Jack environment](https://gym.openai.com/envs/Blackjack-v0/) in this task. [OpenAI Gym](https://gym.openai.com/) is a toolkit for developing and comparing reinforcement learning algorithms. One of its features is to provide various RL-ready environments to facilitate studing and developing new Reinforcement Learning algorithms.

The main purposes of this notebook are to introduce:
- OpenAI Gym environments
- Monte Calro Methods
- the `Exploring starts` exploration algorithm 




In [33]:
#!pip install gym
#!pip install pygame

In [34]:
import gym 
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
from tqdm import tqdm

### Black Jack Environment
- `states` - are provided as tuples (`score`, `dealer score`, `useable ace`)
    - `score` - the summary score of your cards (4-21)
    - `dealer score` - the first dealer card (1-10)
    - `useable ace` - True/False, points out if you have a useable ace
- `actions`
    - `0` - draw
    - `1` - hit
- `rewards`
    - `1` - you won a game
    - `0` - draw
    - `-1` - you lost a game
    
    
Let's create an environment and see how to use it.

In [35]:
env = gym.make('Blackjack-v1')
state, info = env.reset()
print(f'Initial State: {state}.')

Initial State: (15, 5, False).


In [36]:
def sample_policy(state):
    score, dealer_score, usable_ace = state
    return 0 if score >= 19 else 1

state, info = env.reset()
terminal = False
while not terminal:
    action = sample_policy(state)
    
    next_state, reward, terminal, truncated, info = env.step(action)
    print (f'State: {state}, Action: {action}, Next state: {next_state}, Reward: {reward}, Terminal: {terminal}')
    
    state = next_state
    
print (f'Terminal state: {state}, Final reward: {reward}')

State: (16, 7, False), Action: 1, Next state: (24, 7, False), Reward: -1.0, Terminal: True
Terminal state: (24, 7, False), Final reward: -1.0


## Task 1
Fill in the placeholders to complete the `update_qest` method.  You are supposed
to compute an updated state-action value according to the formula:


$q_{n+1} = q_{n} + \frac{1}{n}(G_n-q_n)$

where:
- $q_{n}$ - current estimated state-action value 
- $q_{n+1}$ - new estimated state-action value 
- $G_n$ - return obtained for the explored state and action
- $n$ - number of actions (computed separately for each action type)

In [37]:
class Policy:
    """
    This class is used to learn and maintain the policy. It is being done by
    learning action-value methods. 
    Properties: 
        q_est - a dictionary that stores estimated state-action values for 
                the states
                { state : [value for action0, value for action1] }
        n     - a dictionary that stores how many times each state-action 
                value was updated
                { state : [no. updates for action0, no. updates for action 1]}
    Methods:
        act         - returns a greedy action according to a current policy.
                      The initial policy assumes to hit until the score >= 19.
                      Then it is gradually updated in a learning process
        update_qest - updates a specific state-action value using a given 
                      return
        plot        - visualizes the policy.
    """
    
    def __init__(self):
        
        # Dictionary with { state : [state action value for 0, state action value for 1]  }
        self.q_est = {}        
        self.n = {}
        
    def __initialize_state(self, state):
        state_tuple = tuple(state.items())
        if state_tuple in self.q_est:
            return

        score = state['player'][0]
        should_hit = int (score < 19)
        self.q_est[state_tuple] = [1 - should_hit, should_hit]
        self.n[state_tuple] = [0, 0]
        
        
    def act(self, state):
        """
        Returns a greedy action according to the current policy
        Arguments:
            state - a state obtained from OpenAI Gym Black Jack environment
        Returns:
            action - 0 for 'draw', 1 for 'hit'
        """
        if isinstance(state, dict):
            state_tuple = tuple(state.items())
        else:
            state_tuple = state
        self.__initialize_state(state_tuple)
        return np.argmax(self.q_est[state_tuple])
                
    def update_qest(self, state, action, g):
        """
        Updates state-action value for a specific state and a specific action.
        State-action values are computed as a mean of all returns
        Arguments:
            state  - a state obtained from OpenAI Gym Black Jack environment
            action - 0 for 'draw', 1 for 'hit'
            g      - return that should be used for updating        
        """
        state_tuple = tuple(state.items())
        self.__initialize_state(state)
        
        n_sa = self.n[state_tuple][action]
        q_sa = self.q_est[state_tuple][action]
        # ENTER YOUR CODE HERE. 
        # Update self.n[state][action] and self.q_est[state][action]
        n_new = n_sa + 1
        q_new = q_sa + (1/n_new)*(g - q_sa)
    
        self.n[state][action] = n_new
        self.q_est[state][action] = q_new
        
    def plot(self, useable_ace = True):
        """
        Plots a visualization of current policy. It plots the policy only for the explored
        states. The states that haven't been explored yet, are plotted as 'unknown'
        Arguments:
            useable_ace - True / False. It plots different policies, whether the player
                          has or does not have a useable ace.        
        """
        states = [x for x in policy.q_est.keys() if x[2] == useable_ace]
        rows = max(x[0] for x in states)
        cols = max(x[1] for x in states)
        
        res = -1 * np.ones((rows, cols))
        for state in states:
            res[rows-state[0], state[1]-1] = self.act(state)
            
        fig = plt.figure(figsize = (12, 8))

        cmap = matplotlib.colors.ListedColormap(('white', 'r', 'g'), name = 'My Cmap')

        ax = sns.heatmap(res, linewidth=0.5, cmap = cmap)
        cbar = ax.collections[0].colorbar
        cbar.set_ticks([1, 0, -1])
        cbar.set_ticklabels(['hit', 'draw', 'unknown'])
        ax.set_xticks(np.arange(10) + 1)
        ticks = np.arange(10)
        xticklabels = [f'       {x}' for x in list(ticks+1)]

        plt.xticks(ticks, xticklabels, ha = 'left')
        ticks = np.arange(21)
        yticklabels = [f'{x} ' + (' ' if x < 10 else '') for x in list(ticks+1)[::-1]]
        plt.yticks(ticks, yticklabels, rotation=90, va='top')
        plt.title('Black Jack policy with' + ('' if useable_ace else 'out') + ' a useable ace')

        plt.show()        


In [38]:
policy = Policy()

## Task 2
Complete the generate_episode function by filling in the code placeholders
You are supposed to:
- randomly select the initial action and execute this action 
- select every other action according to the policy and execute it
- update policy for each state-action pair generated during the episode

In [41]:
def generate_episode(policy):
    """
    Generates one episode, and updates state-action values of the policy
    acording to the provided policy.
    """
    states = []
    actions = []
    
    state = env.reset()
    
    # Choose the first action randomly
    action = np.random.choice([0, 1])
    
    states.append((state, action))
    actions.append(action)
    
    # ENTER YOUR CODE HERE
    # Execute the action you chose above
    state, reward, terminal, truncated, info = env.step(action)
    # END OF YOUR CODE
    
    while not terminal:
        # ENTER YOUR CODE HERE
        # Choose an action according to the current policy
        action = policy.act(state)
        # END OF YOUR CODE
        
        states.append(state)
        actions.append(action)
                
        # ENTER YOUR CODE HERE
        # Execute the action you chose above
        state, reward, terminal, truncated, info = env.step(action)
        # END OF YOUR CODE
        
    for state, action in zip(states, actions):
        # ENTER YOUR CODE HERE
        # Update qest
        policy.update_qest(state, action, reward)
        pass 
    
        # END OF YOUR CODE
        
        policy.update_qest(state, action, reward)
    

## Task 3
Generate episodes and learn from them until you learn the optimal policy. You may use `policy.plot()` to visualize your policies.

In [42]:
# ENTER YOUR CODE HERE
env = gym.make('Blackjack-v1')
policy = Policy()
# Generate enough episodes to obtain the optimal policy
for i in tqdm(range (1000000)):
    generate_episode(policy)

  0%|          | 0/1000000 [00:00<?, ?it/s]


AttributeError: 'tuple' object has no attribute 'items'

In [ ]:
policy.plot(False)
policy.plot(True)